<a href="https://colab.research.google.com/github/pythonkvs/seminars/blob/main/homeworks/3/%D0%9A%D0%BB%D0%B8%D0%B5%D0%BD%D1%82_%D0%B4%D0%BB%D1%8F_%D0%BE%D1%82%D0%BF%D1%80%D0%B0%D0%B2%D0%BA%D0%B8_%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA_FAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.coursera.org/learn/diving-in-python/discussions/all/threads/BkiXTtjtEembnxKC4yfdRg

**FAQ. Частые вопросы, возникающие при решении задания "Клиент для отправки метрик"**

**Q1.** Правильно ли я понял из задания, что клиент можно реализовать не асинхронным?

**A1.** Да, синхронный клиент очень даже хорошо будет работать.

**Q2.**  Как реализовать клиент если нет сервера? Тогда получается и сервер для хранения нужно писать по задаче?

**A2.** 1) Вот код простейшего сервера для тестирования метода **get**, который на любые запросы клиента возвращает один и тот же ответ.

In [ ]:
# реализация сервера для тестирования метода get по заданию - Клиент для отправки метрик
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import socket

sock = socket.socket()
sock.bind(('127.0.0.1', 8888))
sock.listen(1)
conn, addr = sock.accept()

print('Соединение установлено:', addr)

# переменная response хранит строку возвращаемую сервером, если вам для
# тестирования клиента необходим другой ответ, измените ее
response = b'ok\npalm.cpu 10.5 1501864247\neardrum.cpu 15.3 1501864259\n\n'

while True:
    data = conn.recv(1024)
    if not data:
        break
    request = data.decode('utf-8')
    print(f'Получен запрос: {ascii(request)}')
    print(f'Отправлен ответ {ascii(response.decode("utf-8"))}')
    conn.send(response)

conn.close()

Напишите вашу реализацию метода **get**  и начинайте тестирование. Метод **get** должен отправить в запросе к серверу строку, полученную в качестве аргумента. Далее необходимо прочитать ответ сервера (обращу ваше внимание, что это бинарная строка), произвести ее разбор и сохранить данные в словарь. Метод **get** должен вернуть этот словарь. Для тестирования сохраните код сервера в файл **server.py** в директории с вашим решением. Запустите в терминале сервер - 

In [ ]:
$ python server.py

Откройте консоль Python и импортируйте класс **Client** из вашего решения.

In [ ]:
$ python3.6 
Python 3.6.6 (default, Aug  9 2018, 19:58:47) 
....
>>> from client import Client

Далее создайте объект клиента и сделайте запрос на сервер методом **get**

In [ ]:
>>> client = Client("127.0.0.1", 8888, timeout=15)
>>> print(client.get("*"))

Убедитесь, что сервер принимает данные от клиента, проверьте правильные ли данные возвращает метод **get**. При необходимости вы можете изменить значение переменной **response** для получения другого ответа от сервера.

Также сервер можно применять для тестирования метода **put**, так как он выводит сообщение о полученных данных от клиента.

2). Вы можете использовать код простейшего echo_server - (https://docs.python.org/3/library/socket.html?highlight=echo server#example). Вам придется самостоятельно написать реализацию логики, для получения полноценного тестового сервера.


**A3.** В конструкторе клиента передается параметр **timeout** - это время ожидания для чтения или для установки соединения?

**Q3.** Время ожидания - https://docs.python.org/3/library/socket.html#notes-on-socket-timeouts

**A4.** Получаю ошибку - "Тест 7.2. Вызов метода **get** экземпляра класса **Client** должен выбрасывать исключение **ClientError** при получении от сервера не корректного ответа". Не ясно, что прилетает клиенту. Точно не **error**, не ошибка сокета, и не пустые данные. Но что еще может остаться?

**Q4.** Тест 7 проверяет, что вызов метода get экземпляра класса **Client** выбрасывает **ClientError** при не верном ответе сервера. Другими словами, если на корректный запрос сервера приходит ответ формате не соответствующем протоколу, описанному в условиях задания, должно выбрасываться исключение **ClientError**. Вы не учли случай, когда строка ответа сервера содержит кроме ожидаемого содержания и другую текстовую информацию. Пример такого ответа: **"some_text\nok\n\n"**. Какие варианты неправильных ответов еще могут быть, вы можете подумать самостоятельно. 

**Q5.** Не проходит тест 6.2: Вызов метода put экземпляра класса **Client** с параметрами: **metric='eardrum.cpu'**, **value=2** отправляет не верный запрос. Значение **timestamp** должно генерироваться с помощью **int(time.time())**. Сервером получено: **'put eardrum.cpu 2 1580844658'** , ожидалось: **'put eardrum.cpu 2 1580844660'** Получается ошибка в том, что на сервере и у меня локально функция **time.time()** возвращает значения с разницей в 2 секунды? Как я могу это пофиксить? Пробовал просто добавлять к своему значению += 2, тогда другие тесты падают по той же причине.

**A5.** Проверьте, что вы правильно инициализируете значение по умолчанию параметра **timestamp** с помощью вызова **int(time.time())** при определении метода **put**. Часто используемая студентами конструкция - 

In [ ]:
def put(self, metric, value, timestamp=int(time.time())):
  ...

работает не правильно, так как инициализация аргументов по-умолчанию происходит 1 раз - до вызова функции. 

**Q6.** Уточните, пожалуйста, названия метрик должны строго соответствовать тем, что приведены в описании протокола или могу принимать другие значения?

**A6.** Названия метрик, приведенные в описании - всего лишь пример. Название метрики может быть произвольным и необязательно содержать два слова, разделенных точкой. 